## 작사가 인공지능 만들기  
lstm 모델을 기반으로 가사를 입력 했을 때 그 다음으로 올 가사를 작성하는 인공지능을 만드는 작업을 했다.
데이터 전처리 전 데이터의 크기는 187088이다.  
  
동일한 파일이 있는지 확인하기 위해, 파일명을 100개만 출력하였다.  
파일이름 중 Kanye_West.txt, kanye.txt, kanye-west.txt 등 내용이 같아 보이는 파일들이 보였다.    
파일의 내용을 확인해보니 하나의 파일에 하나의 노래에 대한 가사가 저장된 것이 아니라    
파일명과 동일한 가수의 여러 노래들이 저장되어 있었다.  
따라서 비슷한 이름의 파일들을 비교하여 내용이 95%이상 일치 한다면 동일한 파일로 간주하고 하나는 제외하는 작업을 추가하였다.    
아래의 코드는 아지트에서 인천 캠퍼스 학우 분의 코드를 참고하였다.  

In [112]:
#데이터 읽어오기
import glob
import os, re

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' #파일경로 지정 

txt_list = glob.glob(txt_file_path)

cnt = 0
for i in txt_list:
    print(i) #파일 100개 출력 
    cnt += 1
    if cnt > 100:
        break

raw_corpus = [] #가사 담을 리스트 정의

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() 
        raw_corpus.extend(raw) #가사 한 줄씩 저장 
        
print("데이터 크기:", len(raw_corpus)) #187088 파일 제거하기 전 데이터 크기 
print("Examples:\n", raw_corpus[:3])

/aiffel/aiffel/lyricist/data/lyrics/leonard-cohen.txt
/aiffel/aiffel/lyricist/data/lyrics/lil-wayne.txt
/aiffel/aiffel/lyricist/data/lyrics/blink-182.txt
/aiffel/aiffel/lyricist/data/lyrics/bruce-springsteen.txt
/aiffel/aiffel/lyricist/data/lyrics/notorious_big.txt
/aiffel/aiffel/lyricist/data/lyrics/beatles.txt
/aiffel/aiffel/lyricist/data/lyrics/bob-dylan.txt
/aiffel/aiffel/lyricist/data/lyrics/alicia-keys.txt
/aiffel/aiffel/lyricist/data/lyrics/dr-seuss.txt
/aiffel/aiffel/lyricist/data/lyrics/radiohead.txt
/aiffel/aiffel/lyricist/data/lyrics/nickelback.txt
/aiffel/aiffel/lyricist/data/lyrics/prince.txt
/aiffel/aiffel/lyricist/data/lyrics/joni-mitchell.txt
/aiffel/aiffel/lyricist/data/lyrics/missy-elliott.txt
/aiffel/aiffel/lyricist/data/lyrics/dj-khaled.txt
/aiffel/aiffel/lyricist/data/lyrics/michael-jackson.txt
/aiffel/aiffel/lyricist/data/lyrics/janisjoplin.txt
/aiffel/aiffel/lyricist/data/lyrics/al-green.txt
/aiffel/aiffel/lyricist/data/lyrics/bruno-mars.txt
/aiffel/aiffel/lyrici

In [93]:
def check_duplicate_lyrics(file1, file2):
    txt1 = []
    txt2 = []
    with open(file1, "r", encoding="utf-8") as f: #file1 한 줄씩 읽어오기
        raw = f.read().splitlines()
        txt1.extend(raw)
    
    with open(file2, "r" ,encoding="utf-8") as f: #file2 한 줄씩 읽어오기
        raw = f.read().splitlines()
        txt2.extend(raw)
    
    txt1 = set(txt1) #txt1 중복 제거
    txt2 = set(txt2) #txt2 중복 제거
    diff = txt1.difference(txt2) #txt2와 다른 가사만 변수 diff에 넣기
    return len(txt1) * 0.05 > len(diff) 

import itertools

for a, b in itertools.combinations(txt_list, 2):#txt_list에서 원소 개수가 2개인 조합 뽑기
    if check_duplicate_lyrics(a, b): #95% 일치한 경우 
        print(a, b)
        txt_list.remove(b)

/aiffel/aiffel/lyricist/data/lyrics/notorious_big.txt /aiffel/aiffel/lyricist/data/lyrics/notorious-big.txt
/aiffel/aiffel/lyricist/data/lyrics/Kanye_West.txt /aiffel/aiffel/lyricist/data/lyrics/kanye-west.txt


notorious_big.txt와 notorious-big.txt의 내용이 95%이상 동일하다고 보고 notorious-big.txt 제외했고  
Kanye_West.txt와 kanye-west.txt의 내용이 95%이상 동일하다고 보고 kanye-west.txt 파일을 데이터에서 제외했다.

In [113]:
raw_corpus = [] #가사 담을 리스트 정의

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() 
        raw_corpus.extend(raw) #가사 한 줄씩 저장 
        
print("데이터 크기:", len(raw_corpus)) 
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [114]:
#정규표현식(Regex)을 이용한 데이터 전처리
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #소문자로 바꾸고 양쪽 공백 지움
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #특수문자 양쪽에 공백 넣기
    sentence = re.sub(r'[" "]+', " ", sentence) #여러 공백 하나의 공백으로 바꿈
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿈
    #sentence = re.sub(r"[^a-zA-Z?.!¿]+", " ", sentence) # a-zA-Z?.!¿가 아닌 모든 문자를 하나의 공백으로 바꿈
    #sentence = re.sub(r"[^a-zA-Z?!,¿]+", " ", sentence) #155886
    #sentence = re.sub(r"[^a-zA-Z¿]+", " ", sentence) # a-zA-Z¿가 아닌 모든 문자를 하나의 공백으로 바꿈 => overfitting 발생 
    sentence = sentence.strip() # 양쪽 공백 지움
    sentence = '<start> ' + sentence + ' <end>' #문장 시작에 start 끝 end 추가 
    return sentence

corpus = [] #전처리 된 데이터 저장 장소

#cnt = 0
for sentence in raw_corpus:
    if len(sentence) == 0:continue #공백 문자이면 건너뛰기
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)   

corpus[:20]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>',
 '<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>',
 '<start> she tied you <end>',
 '<start> to a kitchen chair <end>',
 '<start> she broke your throne , and she cut your hair <end>',
 '<start> and from your lips she drew the hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah you say i took the name in vain <end>',
 '<start> i don t even know the name <end>']

In [116]:
corpus_set = set(corpus) #중복되는 가사 제거 

In [117]:
import tensorflow as tf
from tensorflow import keras

In [118]:
#tokenize:전처리된 단어들을 입력받아 tensor와 tokenizer 반환하는 함수
def tokenize(corpus): 
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,#단어 15000개로 단어 사전 만들기 
        filters=' ', #전처리 했기 때문에 filter 필요없음
        split=' ', #공백 기준으로 단어 분리 
        oov_token="<unk>" #단어 사전에 포함 못하면 <unk>로 대체  
    )

    tokenizer.fit_on_texts(corpus)  
    tensor = tokenizer.texts_to_sequences(corpus) #단어들을 숫자의 시퀀스 형태로 변환
    #tensor = [x for x in tensor if len(x) <= 15]
    
    #padding='post':뒤에 0 붙이기
    #max_len=15:시퀀스 길이 최대 15로 지정 => 토큰 길이가 15를 넘게 되면 잘림 
    #tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post') 
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15) 

    return tensor, tokenizer

#tensor, tokenizer = tokenize(corpus_set) 
tensor, tokenizer = tokenize(corpus) 

In [119]:
tensor.shape 

(175986, 15)

check_duplicate_lyrics 함수거치고 set는 적용하지 않은 경우 tensor shape:(164512, 15), 
                                                           Source Train:(131609, 14), Target Train:(131609, 14)
check_duplicate_lyrics 함수거치고 set는 적용한 경우 tensor shape:(164512, 15)
                                                           Source Train:(131609, 14), Target Train:(131609, 14)  
check_duplicate_lyrics 함수거치지 않고 set는 적용하지 않은 경우 tensor.shape:(175986, 15)  
                                                           Source Train: (140788, 14), Target Train: (140788, 14)  
check_duplicate_lyrics 함수거치지 않고 set는 적용한 경우 tensor.shape:(175986, 15)  
                                                           Source Train: (140788, 14), Target Train: (140788, 14) 

In [120]:
src_input = tensor[:, :-1] #소스 문장
tgt_input = tensor[:, 1:] #타겟 문장 

In [121]:
from sklearn.model_selection import train_test_split
#train data, test data(20%)  분리 
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,       # 데이트셋 비율
                                                          shuffle=True, 
                                                          random_state=42)

print("train dataset:", enc_train.shape)
print("train label:", dec_train.shape)
print("validation dataset:", enc_val.shape)
print("validation label:", dec_val.shape)

train dataset: (140788, 14)
train label: (140788, 14)
validation dataset: (35198, 14)
validation label: (35198, 14)


In [122]:
#데이터 셋 객체 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1 #단어사전 15000개 + 포함 되지 않은 0:<pad> 포함하여 15001개

#training data set 만들기  
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

#validation data set 만들기 
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [123]:
train_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [124]:
val_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [125]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140788, 14)
Target Train: (140788, 14)


In [126]:
#모델 설계
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out

embedding_size = 512 
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [127]:
for src_sample, tft_sample in train_dataset.take(1): break #데이터 셋에서 한 배치만 불러오는 방법 
model(src_sample)

<tf.Tensor: shape=(256, 14, 15001), dtype=float32, numpy=
array([[[-7.26223661e-05,  1.26497151e-04,  4.75626148e-04, ...,
          1.90099483e-04,  9.28074951e-05,  2.82949914e-04],
        [ 1.59770701e-04,  1.29821201e-04,  8.70480610e-04, ...,
          1.86664445e-04,  1.95050729e-04,  6.85614243e-04],
        [ 3.10330011e-04,  1.35859504e-04,  9.54218034e-04, ...,
          1.92492866e-04,  3.99787823e-04,  5.81416127e-04],
        ...,
        [-9.99067794e-04,  2.46352423e-03,  1.94784522e-03, ...,
         -1.49393291e-03,  1.97466323e-03,  6.89141918e-04],
        [-1.11425749e-03,  2.83004669e-03,  2.07325537e-03, ...,
         -1.59573799e-03,  1.96282309e-03,  7.23199511e-04],
        [-1.22180465e-03,  3.15000303e-03,  2.16650101e-03, ...,
         -1.71365717e-03,  1.91523985e-03,  7.11723871e-04]],

       [[-7.26223661e-05,  1.26497151e-04,  4.75626148e-04, ...,
          1.90099483e-04,  9.28074951e-05,  2.82949914e-04],
        [-2.62221110e-05,  5.56075305e-04,  9

In [87]:
model.summary()

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  7680512   
_________________________________________________________________
lstm_6 (LSTM)                multiple                  20979712  
_________________________________________________________________
lstm_7 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense_3 (Dense)              multiple                  30737049  
Total params: 92,959,897
Trainable params: 92,959,897
Non-trainable params: 0
_________________________________________________________________


In [129]:
#model compile
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, #from_logits:모델의 출력값이 문제에 맞게 normalize 된지 여부, 모델의 출력값이 확률이 아닌 값이 나온 경우 true로 지정
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

In [34]:
#case1: 중복된 파일 제거하고 가사간의 중복은 제거 하지 않은 경우 
model.fit(train_dataset, epochs=10) #train data 학습

Epoch 1/10
514/514 [==============================] - 263s 503ms/step - loss: 3.6268
Epoch 2/10
514/514 [==============================] - 267s 519ms/step - loss: 2.9782
Epoch 3/10
514/514 [==============================] - 267s 520ms/step - loss: 2.6765
Epoch 4/10
514/514 [==============================] - 268s 521ms/step - loss: 2.3981
Epoch 5/10
514/514 [==============================] - 267s 520ms/step - loss: 2.1378
Epoch 6/10
514/514 [==============================] - 268s 521ms/step - loss: 1.8952
Epoch 7/10
514/514 [==============================] - 268s 521ms/step - loss: 1.6744
Epoch 8/10
514/514 [==============================] - 268s 521ms/step - loss: 1.4762
Epoch 9/10
514/514 [==============================] - 268s 520ms/step - loss: 1.3016
Epoch 10/10
514/514 [==============================] - 267s 520ms/step - loss: 1.1571


In [55]:
#case2: 중복된 파일 제거하고 가사간의 중복도 set로 제거한 경우
model.fit(train_dataset, epochs=10)

Epoch 1/10
514/514 [==============================] - 265s 512ms/step - loss: 3.6171
Epoch 2/10
514/514 [==============================] - 269s 524ms/step - loss: 3.0371
Epoch 3/10
514/514 [==============================] - 270s 525ms/step - loss: 2.7869
Epoch 4/10
514/514 [==============================] - 270s 526ms/step - loss: 2.5660
Epoch 5/10
514/514 [==============================] - 270s 524ms/step - loss: 2.3594
Epoch 6/10
514/514 [==============================] - 270s 525ms/step - loss: 2.1638
Epoch 7/10
514/514 [==============================] - 269s 523ms/step - loss: 1.9774
Epoch 8/10
514/514 [==============================] - 267s 519ms/step - loss: 1.8019
Epoch 9/10
514/514 [==============================] - 267s 520ms/step - loss: 1.6360
Epoch 10/10
514/514 [==============================] - 267s 520ms/step - loss: 1.4796


In [109]:
#case3: 중복된 파일 제거하지 않고 가사간의 중복도 set로 제거하지 않은 경우
model.fit(train_dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 273s 494ms/step - loss: 3.6714
Epoch 2/10
549/549 [==============================] - 283s 516ms/step - loss: 3.0206
Epoch 3/10
549/549 [==============================] - 284s 517ms/step - loss: 2.7077
Epoch 4/10
549/549 [==============================] - 286s 520ms/step - loss: 2.4157
Epoch 5/10
549/549 [==============================] - 286s 520ms/step - loss: 2.1361
Epoch 6/10
549/549 [==============================] - 286s 521ms/step - loss: 1.8733
Epoch 7/10
549/549 [==============================] - 286s 521ms/step - loss: 1.6343
Epoch 8/10
549/549 [==============================] - 286s 521ms/step - loss: 1.4274
Epoch 9/10
549/549 [==============================] - 286s 521ms/step - loss: 1.2556
Epoch 10/10
549/549 [==============================] - 286s 521ms/step - loss: 1.1215


In [130]:
#case4: 중복된 파일 제거하지 않고 가사간의 중복 set로 제거한 경우
model.fit(train_dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 269s 487ms/step - loss: 3.6998
Epoch 2/10
549/549 [==============================] - 283s 515ms/step - loss: 3.1205
Epoch 3/10
549/549 [==============================] - 285s 519ms/step - loss: 2.8583
Epoch 4/10
549/549 [==============================] - 286s 520ms/step - loss: 2.6329
Epoch 5/10
549/549 [==============================] - 286s 520ms/step - loss: 2.4260
Epoch 6/10
549/549 [==============================] - 286s 521ms/step - loss: 2.2361
Epoch 7/10
549/549 [==============================] - 286s 520ms/step - loss: 2.0577
Epoch 8/10
549/549 [==============================] - 286s 520ms/step - loss: 1.8886
Epoch 9/10
549/549 [==============================] - 286s 520ms/step - loss: 1.7263
Epoch 10/10
549/549 [==============================] - 286s 520ms/step - loss: 1.5714


In [131]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    
    test_input = tokenizer.texts_to_sequences([init_sentence]) #init_sentence 텐서로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64) #test_input을 텐서로 변환 
    end_token = tokenizer.word_index["<end>"] #<end> 단어에 해당하는 키-쌍 딕셔너리 반환 

    while True:
        predict = model(test_tensor) #tensor로 변환된 데이터 입력 받음 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] #예측된 값 중 가장 높은 확률인 word index 뽑아냄 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)#word index를 문장 뒤에 붙임
        
        if predict_word.numpy()[0] == end_token: break #모델이 <end>를 예측
        if test_tensor.shape[1] >= max_len: break #max_len에 도달 

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " " #word index를 단어로 하나씩 변환

    return generated #완성된 문장 반환 

In [132]:
generate_text(model, tokenizer, init_sentence="<start> They said ", max_len=20)

'<start> they said thou shall not be <end> '

## Discussion
<b>모델 설명</b> batch_size는 256, 총 epoch 수는 10, 단어사전 15001개, 토큰 수 15, embedding_size = 512, hidden_size = 2048,  
1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성된 LSTM model  

<b>case1 check_duplicate_lyrics 함수거치고 set는 적용하지 않은 경우</b> embedding_size = 512, hidden_size = 2048, epoch5 일 때 loss가 2.2 이하로 떨어졌고 최종 loss는 1.1571였다. generate_text 함수에 "hallelujah"를 인자로 준 것은 종교적인 이유는 아니고 hallelujah가 들어간 문장이 87개로 꽤 많았기 때문이다. hallelujah는 주로 문장의 끝에 쓰이거나 단독으로 쓰이는 때가 많기 때문에 직관적으로 hallelujah 뒤에는 <end>가 올 것임을 예측할 수 있다. case1의 결과 hallelujah를 입력했을 때 예상대로 <end> 토큰이 나왔다.('<start> hallelujah <end> ')  
  
<b>case2 check_duplicate_lyrics 함수거치고 set는 적용한 경우(중복된 가사 제거)</b> embedding_size = 512, hidden_size = 2048, 최종 loss는 1.4796으로 case1 보다 loss가 더 컸다. generate_text에 "I love" 를 입력했을 때 lstm은 '<start> i love the way you lie <end> '를 작사했는데 음... " i love the way you lie"라는 결과가 잘 받아들여지지 않지만 아마도 가사 중에 이러한 문장이 많았기 때문이지 않을까 싶다.  
    
<b>case3 check_duplicate_lyrics 함수거치지 않고 set는 적용하지 않은 경우</b> embedding_size = 512, hidden_size = 2048, 최종 loss는 1.1215였는데 이는 check_duplicate_lyrics 함수를 거친 case1, 2 보다 낮다. 굳이 중복되는 파일을 삭제하지 않아도 충분히 낮은 loss를 얻은 이유가 무엇일지 궁금하다. 아마도 train data는 반복 학습을 할 수록 overfitting 되는 경향이 증가하니까 loss 값도 중복 되는 데이터가 많을 수록 줄어드는게 아닐까 싶다. generate_text의 결과는 '<start> she said honey if you want you can come with me <end> '로 꽤 도발적인 결과가 나왔다.  
    
<b>case4 check_duplicate_lyrics 함수거치지 않고 set는 적용한 경우</b> embedding_size = 512, hidden_size = 2048, 최종 loss는 1.5714였고 위 세가지 중 가장 loss가 높았다. generate_text의 결과는 '<start> they said thou shall not be <end> '라는 어색한 문장이 출력되었다.  

    